# Predicting blood's geloucos level using LSTM network

## Project Members
* Amir Mohammad Fallah (970122680014)

### Including requirements

In [2]:
from math import sqrt
from numpy import concatenate
import numpy as np
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
import pandas as pd 
import io
import os

Using TensorFlow backend.


### Converting time series to supervised problem

In [4]:
def series_to_supervised(data, n_in=2, n_out=2, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Loading dataset into pandas dataframe

In [50]:
dataframe = pd.read_csv('dataset/data.csv')
# drop missing data
dataframe = dataframe.dropna()
dataframe = dataframe.drop("PtID",axis=1)
values = dataframe.values
values = values.astype('float32')

### Scaling geloucos data range of 0 to 1

In [66]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaledContinues = scaler.fit_transform(dataframe[["Glucose"]])
reframed = series_to_supervised(scaledContinues,1,1)


### Split dataset into train, validation and test sets

In [83]:
values = reframed.values
n_train_len = int(len(values) * 0.7)
n_val_len = int(n_train_len * 0.1)


train = values[:n_train_len, :]
val = train[:n_val_len, :]
test = values[n_train_len:, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
val_X, val_y = val[:, :-1], val[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

### Reshape input to be 3D [samples, timesteps, features]

In [84]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

### Build LSTM network

In [85]:
model = None
# building the model
model = Sequential()
model.add(LSTM(60, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(100, activation='selu'))
model.add(Dense(85, activation='selu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1 ,activation="linear"))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

### Fit network on prepared sets

In [86]:
history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(val_X, val_y), verbose=2, shuffle=False)

Train on 27999 samples, validate on 2799 samples
Epoch 1/100
 - 1s - loss: 0.0052 - accuracy: 0.0341 - val_loss: 0.0022 - val_accuracy: 0.0264
Epoch 2/100
 - 1s - loss: 0.0019 - accuracy: 0.0342 - val_loss: 0.0022 - val_accuracy: 0.0264
Epoch 3/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 4/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 5/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0024 - val_accuracy: 0.0264
Epoch 6/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 7/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 8/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 9/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 10/100
 - 1s - loss: 0.0018 - accuracy: 0.0342 - val_loss: 0.0023 - val_accuracy: 0.0264
Epoch 11/1

Epoch 87/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 88/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 89/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 90/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 91/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 92/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 93/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 94/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 95/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 96/100
 - 1s - loss: 0.0017 - accuracy: 0.0342 - val_loss: 0.0021 - val_accuracy: 0.0264
Epoch 97/100
 - 1s - loss: 0.0017 - accuracy: 0.03